<a href="https://colab.research.google.com/github/GummyBear-w/aop113b/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    ImageMessage,
    VideoMessage,
    LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))


@app.route('/callback', methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


import google.generativeai as genai
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
            # reply stickers
            reply = StickerMessage(
                package_id='446',
                sticker_id='1989'
            )
        elif action == 'image':
            # reply images
            reply = ImageMessage(
                original_content_url='https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg',
                preview_image_url='https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg'
            )
        elif action == 'video':
            reply = VideoMessage(
                original_content_url='https://cdn.pixabay.com/video/2021/07/08/80724-573496156_large.mp4',
                preview_image_url='https://cdn.pixabay.com/photo/2017/02/11/14/45/taipei-taiwan-2057818_1280.jpg'
            )
        elif action == 'location':
            reply = LocationMessage(
                title='Taipei 101',
                address='110台北市信義區信義路五段7號',
                latitude=25.034199559170606,
                longitude=121.56456035396677
            )
        else:
            response = ask_gemini(action)
            reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )


ngrok_start()
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5676-35-234-34-7.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:03:44] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:03:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:03:58] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:04:06] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:04:26] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:05:00] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:05:07] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:05:14] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 08:05:19] "POST /callback HTTP/1.1" 200 -
